# Epoching + TFRs for all subjects


*Created: 09/14/2024* \
*Updated: 09/26/2024* 

- SHORTER EPOCHS FOR COH DATA
- no baselining/ch removal - just epochs fif creation

In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
from ast import literal_eval



import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/sc/arion/projects/guLab/Alie/SWB/ephys_analysis/LFPAnalysis/')

In [4]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils, nlx_utils

In [5]:
# Specify root directory for un-archived data and results 
base_dir   = '/sc/arion/projects/guLab/Alie/SWB/'
anat_dir   = f'{base_dir}ephys_analysis/recon_labels/'
neural_dir = f'{base_dir}ephys_analysis/data/'
behav_dir  = f'{base_dir}ephys_analysis/behav/data/'

date = datetime.date.today().strftime('%m%d%Y')
print(date)

09262024


## Gather All Subj Info

In [6]:
# Subj list for CCN 2024
subj_ids = list(pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
n_subj = len(subj_ids)
subj_ids


['MS002',
 'MS003',
 'MS009',
 'MS011',
 'MS015',
 'MS016',
 'MS017',
 'MS019',
 'MS022',
 'MS024',
 'MS025',
 'MS026',
 'MS027',
 'MS028',
 'MS029',
 'MS030',
 'MS033',
 'MS035',
 'MS041',
 'MS043',
 'MS048',
 'MS050',
 'DA8',
 'DA023',
 'DA026',
 'DA037',
 'DA039']

In [7]:
anat_info_all_subj = {f'{subj_id}': pd.read_csv(f'{anat_dir}{subj_id}_labels.csv') for subj_id in subj_ids}


In [8]:
anat_info_all_subj

{'MS002':        label BN246label          x          y          z      mni_x  \
 0     LaCaS1    A32sg_L  -6.784557  49.555827   0.149337  -6.382462   
 1    LaCaS10      A9l_L  -8.380923  61.155821  43.676991  -8.220073   
 2    LaCaS11      A9l_L  -8.780014  62.355821  48.469026  -8.542609   
 3    LaCaS12    Unknown  -8.780014  63.555820  53.261062  -8.436667   
 4     LaCaS2    A32sg_L  -6.784557  50.755827   4.941372  -6.368174   
 ..       ...        ...        ...        ...        ...        ...   
 153   RmOIF5  A12/47o_R  33.922762  42.355831  -8.636061  34.666463   
 154   RmOIF6  A12/47l_R  37.913675  41.555831  -6.240044  38.716568   
 155   RmOIF7  A12/47l_R  41.904589  40.755832  -3.844026  42.794042   
 156   RmOIF8  A12/47l_R  46.294594  39.555832  -1.048672  47.308597   
 157   RmOIF9     A45r_R  50.285508  38.755833   1.746682  51.450368   
 
          mni_y      mni_z       gm  \
 0    37.158688  -3.130044     Gray   
 1    48.757099  44.868611     Gray   
 2    49

In [9]:
anat_info_all_subj.keys()

dict_keys(['MS002', 'MS003', 'MS009', 'MS011', 'MS015', 'MS016', 'MS017', 'MS019', 'MS022', 'MS024', 'MS025', 'MS026', 'MS027', 'MS028', 'MS029', 'MS030', 'MS033', 'MS035', 'MS041', 'MS043', 'MS048', 'MS050', 'DA8', 'DA023', 'DA026', 'DA037', 'DA039'])

# Create and Save Epoched Data for all subjects

In [12]:
# Set epoch for connectivity 
epoch_id  = 'CpeOnset'
epoch_dur = [0,1.0]

# IED removal requires that we set some parameters for IED detection. 
# 1. peak_thresh: how many stds should they exceed the baseline by? 
# 2. closeness_thresh: how close should they be allowed to be (in sec) to other candidate IEDs? 
# 3. width_thresh: how wide should they have to be (in sec)?

# Defaults:
IED_args = {'peak_thresh':4,
           'closeness_thresh':0.25, 
           'width_thresh':0.2}

# Define spectrogram parameters for morlet wavelets 

freqs = np.logspace(*np.log10([2, 200]), num=30)
n_cycles = np.floor(np.logspace(*np.log10([3, 10]), num=30))


In [15]:

for subj_id in subj_ids: 
    
    # load photodiode data
    diode_info_fname  = f'{neural_dir}diode_info/{subj_id}_diode_info.npy'
    slope,offset = np.load(diode_info_fname)
    # load behav data 
    behav_df = pd.read_csv(f'{behav_dir}{subj_id}_task_df.csv')
    # update behav_df to have a cpe timestamp 
    behav_df['CpeOnset'] = behav_df.DecisionOnset + 2.0
    
    # Make the epochs. 
    ev_epochs = lfp_preprocess_utils.make_epochs(load_path=f'{neural_dir}{subj_id}/bp_ref_ieeg.fif', 
                                                 slope=slope, offset=offset,
                                                 behav_name=epoch_id, behav_times=behav_df[epoch_id].values,
                                                 ev_start_s=epoch_dur[0], ev_end_s=epoch_dur[1],
                                                 buf_s = 1, IED_args=IED_args,
                                                 downsamp_factor=None,  
                                                 detrend=0) #### new - constant detrend by DC offset https://mne.tools/stable/generated/mne.Epochs.html

    ev_epochs.save(f'{neural_dir}{subj_id}/{subj_id}_conn_epochs_{epoch_id}.fif', overwrite=True)

    
    


Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/bp_ref_ieeg.fif...
    Range : 0 ... 1083499 =      0.000 ...  2166.998 secs
Ready.
Reading 0 ... 1083499  =      0.000 ...  2166.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 265 samples (0.530 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:   15.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:   12.1s finished


Used Annotations descriptions: ['CpeOnset']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 1501 original time points ...
0 bad epochs dropped
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS003/bp_ref_ieeg.fif...
    Range : 0 ... 1250999 =      0.000 ...  2501.998 secs
Ready.
Reading 0 ... 1250999  =      0.000 ...  2501.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
-

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  83 out of  83 | elapsed:    5.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  83 out of  83 | elapsed:    5.9s finished


Used Annotations descriptions: ['CpeOnset']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 1501 original time points ...
0 bad epochs dropped
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS009/bp_ref_ieeg.fif...
    Range : 0 ... 1077655 =      0.000 ...  2155.310 secs
Ready.
Reading 0 ... 1077655  =      0.000 ...  2155.310 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
-

...

Used Annotations descriptions: ['CpeOnset']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 1501 original time points ...
0 bad epochs dropped
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/DA037/bp_ref_ieeg.fif...
    Range : 0 ... 1150155 =      0.000 ...  2300.310 secs
Ready.
Reading 0 ... 1150155  =      0.000 ...  2300.310 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
-

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    1.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    1.7s finished


Used Annotations descriptions: ['CpeOnset']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 1501 original time points ...
0 bad epochs dropped
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/DA039/bp_ref_ieeg.fif...
    Range : 0 ... 1165061 =      0.000 ...  2330.122 secs
Ready.
Reading 0 ... 1165061  =      0.000 ...  2330.122 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
-

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:    3.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:    5.0s finished


Used Annotations descriptions: ['CpeOnset']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 1501 original time points ...
0 bad epochs dropped


In [ ]:
# #### update all subjects epochs data and resave 
# epoch_id = 'CpeOnset'
# # iterate through subjects
# for subj_id in subj_ids:
#     print(subj_id)
#     # load & format rereferenced epoch data 
#     subj_epochs = mne.read_epochs(f'{neural_dir}{subj_id}/{epoch_id}_epochs.fif', preload=True)
#     # drop bad trials
#     subj_drops = drops_data[subj_id]
#     subj_epochs.drop(subj_drops)
#     # replace old metadata with updated subject data
#     subj_epochs.metadata = all_behav[all_behav.subj_id == subj_id]
#     # save updated epochs data
#     subj_epochs.save(f'{neural_dir}{subj_id}/{epoch_id}_epochs-clean.fif', overwrite=True)
#     del subj_epochs

In [24]:
#### Set global vars for all subj 


# baseline_ev  = 'TrialOnset' # fixation cross baseline epoch
# analysis_evs = ['CpeOnset']

# # dictionary of event ids and times - repeated from above
# evs = {'TrialOnset': [-1.0, 0], #ITI is always at least one second + 10ms buffer for wavelet padding (no horizontal smearing)
#        'CpeOnset': [0,3.0] } # 3 sec following CPE outcome reveal - timestamps are made manually above


In [25]:
# # IED removal requires that we set some parameters for IED detection. 
# # 1. peak_thresh: how many stds should they exceed the baseline by? 
# # 2. closeness_thresh: how close should they be allowed to be (in sec) to other candidate IEDs? 
# # 3. width_thresh: how wide should they have to be (in sec)?

# # Defaults:
# IED_args = {'peak_thresh':4,
#            'closeness_thresh':0.25, 
#            'width_thresh':0.2}

In [26]:
# # Define spectrogram parameters for morlet wavelets 

# freqs = np.logspace(*np.log10([2, 200]), num=30)
# n_cycles = np.floor(np.logspace(*np.log10([3, 10]), num=30))


In [11]:
# # for every subject - 


# for subj_id in subj_ids: 
#     print(f'start epoching {subj_id}')
    
#     # load photodiode data
#     diode_info_fname  = f'{neural_dir}diode_info/{subj_id}_diode_info.npy'
#     slope,offset = np.load(diode_info_fname)
#     # load behav data 
#     behav_df = pd.read_csv(f'{behav_dir}{subj_id}_task_df.csv')
#     # update behav_df to have a cpe timestamp 
#     behav_df['CpeOnset'] = behav_df.DecisionOnset + 2.0
    
    
#     epochs_all_evs = {f'{x}': np.nan for x in evs}
    
#     ### make epochs should now find IEDs, artifact times, and save info as df in load_path directory for every ev

#     for event in evs.keys():
#         # Make the epochs. 
#         ev_epochs = lfp_preprocess_utils.make_epochs(load_path=f'{neural_dir}{subj_id}/bp_ref_ieeg.fif', 
#                                                      slope=slope, offset=offset,
#                                                      behav_name=event, behav_times=behav_df[event].values,
#                                                      ev_start_s=evs[event][0], ev_end_s=evs[event][1],
#                                                      buf_s = 1, IED_args=IED_args,
#                                                      downsamp_factor=None,  
#                                                      detrend=0) #### new - constant detrend by DC offset https://mne.tools/stable/generated/mne.Epochs.html

#         epochs_all_evs[event] = ev_epochs
#         ### save epoched data 
#         ev_epochs.save(f'{neural_dir}{subj_id}/{event}_epochs.fif', overwrite=True)
    
    
# #     #### COMPUTE BASELINE TFRs FOR ONE SUBJ
# #     # baseline epoch dictionary 
# #     epochs_all_baseline = {} 

# #     # baseline event = TrialOnset epochs 
# #     baseline_epochs_reref = epochs_all_evs[baseline_ev]

# #     # dropping bad channels from baseline epochs 
# #     baseline_epochs_reref = baseline_epochs_reref.drop_channels(baseline_epochs_reref.info['bads'])

# #     # compute baseline tfr 
# #     temp_pow  = mne.time_frequency.tfr_morlet(baseline_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
# #                                               picks=baseline_epochs_reref.ch_names, # now doesn't include bad ch
# #                                               use_fft=True, n_jobs=-1, output='power',
# #                                               return_itc=False, average=False) # returns pow (not ITC) for all epochs

# #     temp_pow.crop(tmin=evs[baseline_ev][0], tmax=evs[baseline_ev][1]) # clipping epoch to remove buffer

# #     # Remove bad data by naning out +/-100ms around bad timepoints (IEDs and artifacts)
# #     # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
# #     IED_df = pd.read_csv(f'{neural_dir}{subj_id}/{baseline_ev}_IED_df.csv') 
# #     artifact_df = pd.read_csv(f'{neural_dir}{subj_id}/{baseline_ev}_artifact_df.csv') 

# #     # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
# #     for ch_ix, ch_name in enumerate(baseline_epochs_reref.ch_names):
# #         # lists of IED/artifact timestamps from dfs for one channel (if any)
# #         ied_ev_list = IED_df[ch_name].dropna().index.tolist()
# #         artifact_ev_list = artifact_df[ch_name].dropna().index.tolist()

# #         # iterate through every IED timestamp in channel 
# #         for ev_ in ied_ev_list: 
# #             # iterate through values in IED df (literal eval = evaluates string as literal python data structures)
# #             for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
# #                 # remove 100 ms before 
# #                 ev_ix_start = np.max([0, np.floor((ied_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
# #                 # remove 100 ms after 
# #                 ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
# #                 temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

# #         # iterate through every artifact timestamp in channel 
# #         for ev_ in artifact_ev_list: 
# #              # iterate through values in artifact df (literal eval = evaluates string as literal python data structures)
# #             for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
# #                 # remove 100 ms before 
# #                 ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
# #                 # remove 100 ms after
# #                 ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
# #                 temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

# #     epochs_all_baseline[baseline_ev] = temp_pow


# #     ##### COMPUTE TFR FOR ANALYSIS EPOCHS AND BASELINE TO BASELINE TFR DATA

# #         # repeat baselining procedure for every event in analysis_evs
# #         # remove +/-100ms around IED/artifact individually for each analysis epoch 
# #         # baseline analysis ev tfrs
# #         # iteratively check for zscores still above 10 std, remove those timepoints, then recompute baselined tfr


# #     power_epochs = {} # keys = analysis evs 

# #     for event in analysis_evs: 
# #         print('Compute baselined TFR for epoch: ',event)

# #         # extract raw epoch data for analysis ev 
# #         event_epochs_reref = epochs_all_evs[event].copy()
# #         # dropping bad channels from baseline epochs 
# #         event_epochs_reref = event_epochs_reref.drop_channels(event_epochs_reref.info['bads'])
# #         ch_names = event_epochs_reref.ch_names

# #         # compute TFR from raw epoched data
# #         temp_pow = mne.time_frequency.tfr_morlet(event_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
# #                                                  picks=event_epochs_reref.ch_names, 
# #                                                  use_fft=True, n_jobs=-1, output='power', 
# #                                                  return_itc=False, average=False)
# #         # crop 1.0 sec buffer from analysis epoch 
# #         temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1])

# #         ### repeat IED/artifact removal procedure used for baseline epochs, but with analysis event IED/artifact times

# #         # load IED/artifact dfs specific to event in analysis evs
# #         IED_df = pd.read_csv(f'{neural_dir}{subj_id}/{event}_IED_df.csv') 
# #         artifact_df = pd.read_csv(f'{neural_dir}{subj_id}/{event}_artifact_df.csv') 

# #         # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
# #         for ch_ix, ch_name in enumerate(event_epochs_reref.ch_names): 
# #             ied_ev_list = IED_df[ch_name].dropna().index.tolist()
# #             artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
# #             for ev_ in ied_ev_list: 
# #                 for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
# #                     # remove 100 ms before 
# #                     ev_ix_start = np.max([0, np.floor((ied_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
# #                     # remove 100 ms after
# #                     ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
# #                     temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
# #             for ev_ in artifact_ev_list: 
# #                 for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
# #                     # remove 100 ms before 
# #                     ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
# #                     # remove 100 ms after
# #                     ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
# #                     temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

# #         ##### remove extreme power values (z>10) remaining after IED/artifact removal 
# #         # iterative process - baseline correct event TFR, remove z > 10, re-baseline until no z>10 or reached max iters


# #         # Let's nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
# #         absurdity_threshold = 10 # zscore threshold for removal
# #         max_iter = 10 # max number of iterations for while loop 
# #         large_z_flag=True # flag that indicates whether large zs are present, when all z<10 = False in while loop

# #         iteration = 0 # intialize iter counter
# #         while (large_z_flag==True) & (iteration<max_iter): # while z>10 still in tfr or under max iter num 
# #             print(f'baseline z-score iteration # {iteration}')
# #             # Baseline by all the baseline periods in the session
# #             baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, 
# #                                   baseline_mne=epochs_all_baseline[baseline_ev].data,  
# #                                   mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)

# #             large_z_mask = np.where(baseline_corrected_power>absurdity_threshold)
# #             if large_z_mask[0].shape[0] == 0:
# #                 # no more large z
# #                 large_z_flag = False
# #             else:
# #                 # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
# #                 # contamination of all z's
# #                 temp_pow.data[large_z_mask] = np.nan

# #             iteration +=1

# #         # input baseline corrected pow into mne EpochsTFR object once all z>10 removed or iteratively removed 10x 
# #         zpow = mne.time_frequency.EpochsTFR(event_epochs_reref.info, baseline_corrected_power,
# #                                             temp_pow.times, freqs)

# #         # add metadata to baseline corrected TFR event epochs object 
# #         zpow.metadata = event_epochs_reref.metadata

# #         # save data (#conda install -c conda-forge h5io must install to save tfr data)
# #         zpow.save(f'{neural_dir}{subj_id}/{subj_id}_{event}-tfr.h5', overwrite=True) 

# #         # store event zpow epochs in dict for all analysis evs 
# #         power_epochs[event] = zpow
        
# #         print(f'finished epoching {subj_id}')
        
# #         del power_epochs, zpow, temp_pow, large_z_mask, event_epochs_reref, ev_epochs, epochs_all_evs, epochs_all_baseline






# Plot TFRs 
- For each analysis event, plot the mean TFR across all epochs for all channels
- Save plots
- Visually inspect TFRs to see if further data cleaning is necessary

In [233]:
# ####add directory for TFRs
# tfr_dir = f'{base_dir}ephys_analysis/figs/single_elec_tfrs/{subj_id}/'

# # print date of TFR plotting 
# date = datetime.date.today().strftime('%m%d%Y')
# print(date, offset) # print date, double check current slope/offset


05102024 17.49815119693244


In [ ]:
# #### for each analysis event - plot one TFR averaged across all epochs for every electrode & save 

# for event in analysis_evs: 
#     print('Plot TFR for epoch: ',event)
#     save_path = f'{tfr_dir}{event}/'
#     os.makedirs(save_path,exist_ok = True) #added so you don't have to manually make folders
    
#     ev_data = power_epochs[event].copy() # tfr plot data - epochs x num ch x freqs x times 
#     times = ev_data.times.copy() # same for all elec
    
#     for ch in ev_data.ch_names:
#         # single ch tfr 
#         plot_data = np.nanmean(np.nanmean(ev_data.copy().pick_channels([ch]).data, axis=0), axis=0) 
        
#         fig, ax = plt.subplots(1, 1, figsize=(6, 4),dpi=300)
        
#         im = ax.imshow(plot_data,interpolation='Bicubic',
#                        aspect='auto', origin='lower', cmap='RdBu_r')
    

#         ax.set(xlabel='Time (s)', ylabel='Frequency (Hz)',title=f'{subj_id}_{ch}_{event}_{date}',
#                yticks=np.arange(0,len(freqs))[::5], 
#                yticklabels=freqs[::5].astype(int), 
#                xticks = np.arange(plot_data.shape[1])[::250],
#                   xticklabels= times[::250])
        
#         fig.colorbar(im, ax=ax)

#         plt.savefig(f'{save_path}{subj_id}_{ch}_{event}_{date}.png', dpi=300)
# #         bbox_inches=None, pad_inches=0.1,
# #         facecolor='auto', edgecolor='auto',backend=None)
#         plt.close()

    

Plot TFR for epoch:  DecisionOnset


In [10]:
# # from salman - faster code

# def compute_and_baseline_tfr(baseline_event, task_events, freqs, n_cycles, load_path, save_path,
#                             IED_artifact_thresh=True, uncaptured_z_thresh=True, output='save'):
    
#     """
    
#     """
    
    
#     baseline_name = list(baseline_event.keys())[0]
    
#     # load baseline epochs
#     baseline_epochs_reref = mne.read_epochs(f'{load_path}/{baseline_name}-epo.fif', preload=True)
    
#     # compute TFR
#     baseline_power  = mne.time_frequency.tfr_morlet(baseline_epochs_reref, 
#                                           freqs=freqs, 
#                                           n_cycles=n_cycles, 
#                                           picks=baseline_epochs_reref.ch_names,
#                                           use_fft=True, 
#                                           n_jobs=-1, 
#                                           output='power', 
#                                           return_itc=False, 
#                                           average=False)
    

#     # Crop the data to the appropriate 
#     baseline_power.crop(tmin=baseline_event[baseline_name][0], 
#                         tmax=baseline_event[baseline_name][1])
    
    
#     if IED_artifact_thresh:
#         # NAN out the bad data
#         # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
#         IED_df = pd.read_csv(f'{load_path}/{baseline_name}_IED_df.csv') 
#         artifact_df = pd.read_csv(f'{load_path}/{baseline_name}_artifact_df.csv') 


#         # Now, let's iterate through each channel, and each ied/artifact, and NaN 100 ms before and after these timepoints
#         for ch_ix, ch_name in enumerate(baseline_epochs_reref.ch_names): 
#             ied_ev_list = IED_df[ch_name].dropna().index.tolist()
#             artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
#             for ev_ in ied_ev_list: 
#                 for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
#                     # remove 100 ms before 
#                     ev_ix_start = np.max([0, np.floor((ied_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     # remove 100 ms after 
#                     ev_ix_end = np.min([baseline_power.data.shape[-1], np.ceil((ied_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     baseline_power.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
#             for ev_ in artifact_ev_list: 
#                 for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
#                     # remove 100 ms before 
#                     ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     # remove 100 ms after
#                     ev_ix_end = np.min([baseline_power.data.shape[-1], np.ceil((artifact_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     baseline_power.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
    
#     # remove epochs from memory
#     del baseline_epochs_reref
    
#     # Now we will deal with the task events of interest   
    
#     # make the output path for the z-scored TFRs
#     if not os.path.exists(save_path):
#         os.makedirs(save_path)
        
#     for event in task_events.keys():
        
#         event_epochs_reref = mne.read_epochs(f'{load_path}/{event}-epo.fif', preload=True)
                    
#         temp_pow = mne.time_frequency.tfr_morlet(event_epochs_reref, 
#                                                  freqs=freqs, 
#                                                  n_cycles=n_cycles,
#                                                  picks=event_epochs_reref.ch_names, 
#                                                  use_fft=True, 
#                                                  n_jobs=-1, 
#                                                  output='power', 
#                                                  return_itc=False, 
#                                                  average=False)
    
#         temp_pow.crop(tmin=task_events[event][0], tmax=task_events[event][1])
        
#         if IED_artifact_thresh:
#             # NAN out the bad data
#             # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
#             IED_df = pd.read_csv(f'{load_path}/{event}_IED_df.csv') 
#             artifact_df = pd.read_csv(f'{load_path}/{event}_artifact_df.csv') 

#             # Now, let's iterate through each channel, and each ied/artifact, and NaN 100 ms before and after these timepoints
#             for ch_ix, ch_name in enumerate(event_epochs_reref.ch_names): 
#                 ied_ev_list = IED_df[ch_name].dropna().index.tolist()
#                 artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
#                 for ev_ in ied_ev_list: 
#                     for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
#                         # remove 100 ms before 
#                         ev_ix_start = np.max([0, np.floor((ied_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         # remove 100 ms after
#                         ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
#                 for ev_ in artifact_ev_list: 
#                     for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
#                         # remove 100 ms before 
#                         ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         # remove 100 ms after
#                         ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan    
    
#         # Compute first pass of baseline
#         baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, include_epoch_in_baseline=False, 
#                                                                        baseline_mne=baseline_power.data, 
#                                                                        mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)
            
#         # Let's iteratively nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
#         if uncaptured_z_thresh: 
#             absurdity_threshold = 10
#             max_iter = 10
#             large_z_flag=True 

#             iteration = 0

#             while (large_z_flag==True) & (iteration<max_iter): 
#                 print(f'baseline z-score iteration # {iteration}')
#                 # Baseline by all the baseline periods in the session
#                 baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, include_epoch_in_baseline=False, 
#                                       baseline_mne=baseline_power.data, 
#                                       mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)

#                 large_z_mask = np.where(np.abs(baseline_corrected_power)>absurdity_threshold)
#                 if large_z_mask[0].shape[0] == 0:
#                     # no more large z
#                     large_z_flag = False
#                 else:
#                     # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
#                     # contamination of all z's
#                     temp_pow.data[large_z_mask] = np.nan

#                 iteration +=1
        
#         zpow = mne.time_frequency.EpochsTFR(event_epochs_reref.info, baseline_corrected_power, 
#                                     temp_pow.times, freqs)

#         zpow.metadata = event_epochs_reref.metadata
        
#         if output == 'save':
#             zpow.save(f'{save_path}/{event}-tfr.h5', overwrite=True)
#         elif output == 'return': 
#             return zpow 
#         elif output == 'both':
#             zpow.save(f'{save_path}/{event}-tfr.h5', overwrite=True)
#             return zpow 
        
# 12:32
# Usage example:
# # Explicitly define a list of analysis events and the baseline event. Should correspond to the dict

# baseline_event = {'baseline_start':[0, 0.75]}
# task_events = {'gamble_start':[-1.0, 0], 
#        'feedback_start': [0, 1.5]}

# day = 'Day1'

# # Define parameters for TFR: 
# freqs = np.logspace(*np.log10([2, 200]), num=30)
# n_cycles = np.floor(np.logspace(*np.log10([3, 10]), num=30))

# subj_id = 'MS012'

# compute_and_baseline_tfr(baseline_event, task_events, freqs, n_cycles, 
#                          load_path=f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/neural/{day}', 
#                          save_path=f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/scratch/TFR',
#                          IED_artifact_thresh=True, uncaptured_z_thresh=True, output='save')